# AdaBoost 분류 기본코드

Adaptive Boost

초기 모형을 약한 모형으로 설정하며 매 스탭마다 가중치를 이용하여 이전 모형의 약점을 보완하는 방식으로 새로운 모형을 순차적으로 학습하고 최종적으로 이들을 선형 결합하여 얻어진 모형을 생성하는 알고리즘

## #01. 준비작업

### [1] 패키지 가져오기


In [1]:
# 연결된 모듈이 업데이트 되면 즉시 자동 로드함
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.analysis import *
from hossam.classification import *

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## #02. 데이터 가져오기


In [2]:
origin = my_read_excel(
    "https://data.hossam.kr/mldata/pima_indians_diabetes.xlsx",
    categories=["Outcome"],
    info=False,
)


상위 5개 행
+----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------+
|    |   Pregnancies |   Glucose |   BloodPressure |   SkinThickness |   Insulin |   BMI |   DiabetesPedigreeFunction |   Age |   Outcome |
|----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------|
|  0 |             6 |       148 |              72 |              35 |         0 |  33.6 |                      0.627 |    50 |         1 |
|  1 |             1 |        85 |              66 |              29 |         0 |  26.6 |                      0.351 |    31 |         0 |
|  2 |             8 |       183 |              64 |               0 |         0 |  23.3 |                      0.672 |    32 |         1 |
|  3 |             1 |        89 |              66 |              23 |        94 |  28.1 |                      0.167 |    21 |         0 |
|  4 |     

### [3] 전처리

`0` 값을 포함해서는 안되는 변수에 대해 평균으로 대체


In [3]:
# 정제 대상 필드명
zero_features = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]

# 0값을 결측치로 대체후 평균으로 다시 치환
df = origin.copy()
df[zero_features] = df[zero_features].replace(0, np.nan)
df2 = my_replace_missing_value(df)

my_pretty_table(df2.head())

+----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------+
|    |   Pregnancies |   Glucose |   BloodPressure |   SkinThickness |   Insulin |   BMI |   DiabetesPedigreeFunction |   Age |   Outcome |
|----+---------------+-----------+-----------------+-----------------+-----------+-------+----------------------------+-------+-----------|
|  0 |             6 |       148 |              72 |              35 |   155.548 |  33.6 |                      0.627 |    50 |         1 |
|  1 |             1 |        85 |              66 |              29 |   155.548 |  26.6 |                      0.351 |    31 |         0 |
|  2 |             8 |       183 |              64 |         29.1534 |   155.548 |  23.3 |                      0.672 |    32 |         1 |
|  3 |             1 |        89 |              66 |              23 |        94 |  28.1 |                      0.167 |    21 |         0 |
|  4 |             0

### [4] 훈련/검증 데이터 분할 및 데이터 표준화


In [4]:
x_train, x_test, y_train, y_test = my_train_test_split(df2, "Outcome", scalling=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

## #02. AdaBoost

### [1] 부스팅에 사용할 최적의 알고리즘 선정


In [5]:
estimator = my_classification(x_train, y_train, x_test, y_test,learning_curve=False)
best_estimator = estimator['best']
best_estimator

가지치기를 하지 않습니다.


### [2] AdaBoost 설정

| 하이퍼 파라미터    | 기본값                | 설명                                        | 추천사항                           |
|----------------|--------------------|-------------------------------------------|---------------------------------|
| `base_estimator` | `None` (DecisionTreeClassifier) | 사용할 약한 학습기. `None` 시 최대 깊이 1의 결정 트리 사용. | 복잡도에 맞게 모델 선택 가능.        |
| `n_estimators` | `50` | 약한 학습기의 최대 개수.                       | 과적합 주의하며 증가 가능.           |
| `learning_rate` | `1.0` | 학습률.                                     | 낮은 학습률과 더 많은 학습기의 조합 권장. |
| `algorithm` | `SAMME.R` | 부스팅 알고리즘. ( `SAMME` 또는 `SAMME.R` )          | `SAMME` 추천.                     |
| `random_state` | `None` | 난수 시드.                                   | 재현 가능한 결과를 위해 설정.         |

.

> `algorithm` 의 경우 현재 버전에서는 `SAMME.R` 이 기본값이지만 다음 버전에서 이 값이 삭제될 것이라는 경고 메시지가 표시되므로 `SAMME` 를 추천함


In [ ]:
ada = AdaBoostClassifier(
    estimator=best_estimator,
    n_estimators=100,
    learning_rate=0.1,
    algorithm='SAMME',
    random_state=get_random_state()    
)

ada.fit(x_train, y_train)

my_classification_result(ada, x_train, y_train, x_test, y_test)
my_classification_report(ada, x_train, y_train, x_test, y_test)

NameError: name 'best_estimator' is not defined